In [25]:
import requests  # 서버에 http 프로토콜로 요청
from bs4 import BeautifulSoup as bs  # html 파싱
import time  # 5초간 휴식 할 때 사용
from datetime import datetime  # 오늘 연월일 추출
import pandas as pd  # 데이터 프레임 만듬
import sqlite3
from sqlalchemy import create_engine
import yes24_dbio as ydb

In [26]:
today = datetime.today()
yyyy = today.year
mm = today.month
print(yyyy, mm)

2023 12


In [27]:
year = list(range(2023, 2022, -1))
month = list(range(1, 13))
page = list(range(1, 10))
print(year)
print(month)
print(page)

[2023]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [13]:
result = []
for yy in year:
    for mon in month if yy != yyyy else range(1, mm)[:1]:
        for p in page[:2]:
            print(f"{yy}년{mon}월{p}페이지 수집중", end="\r")
            url = "https://www.yes24.com/Product/Category/MonthWeekBestSeller"
            payload = dict(categoryNumber='001', pageNumber=p, pageSize='120', type='month', saleYear=yy, saleMonth=mon)          
            
            try:
                r = requests.get(url, params=payload) # yes24 서버에 url로 자료 요청
            except Exception as e:
                print(e)
                
            soup = bs(r.text, 'lxml')   # yes24 서버에서 받은 응답 값 텍스트를 beautifulsoup을 통해 html로 파싱
            book_list = soup.select("#yesBestList > li")
            result.append(ydb.extract_bookinfo(yy, mon, book_list))  # extract_bookinfo 함수로 추출한 데이터를 r
            time.sleep(5)
df2 = pd.DataFrame(result, columns=['yy', 'mon', 'title_h', 'title_f', 'title_m', 'title_e', 'detail_link', \
                        'author', 'publisher', 'pub_date', 'price', 'n_reviews', \
                         'review_link', 'rating', 'tags'])
df2.to_csv("./data/new_yes24bestseller2023_01_1000.csv", index=False)
data2 = pd.read_csv("./data/new_yes24bestseller2023_01_1000.csv")
urls = data2['detail_link']
detail_page_data = ym.detail_page_info(urls)  # detail_page_info 함수로 상세페이지 데이터 추출
df2['book_id'] = df2['detail_link'].apply(lambda x: x.split("/")[-1]) 
final_result_df = pd.merge(df2, detail_page_data, how='inner', on="book_id") # 베스트 셀러 목록과 detail info 합침
final_result_df.head(2)
ym.to_db(final_result_df) # 최종 데이터를 sqlite3 데이터 베이스에 저장

In [60]:
[i.get_text() for i in soup.select(".info_row.info_tag > .tag > a")]

['#대장금_출판사대표추천도서',
 '#독서지도안',
 '#북클러버의선택',
 '#2022아침독서추천도서',
 '#내맘대로올해의책',
 '#2023서울국제도서전',
 '#YES24XEBS_대국민독서챌린지',
 '#2022내맘대로올해의책',
 '#시읽기',
 '#책읽아웃',
 '#세계사',
 '#학습만화',
 '#초등세계사',
 '#편집자의시대_편집자가만든책',
 '#크레마클럽에있어요',
 '#돈좀벌어보자',
 '#돈이란무엇인가',
 '#돈구경시켜드릴게',
 '#애니원작',
 '#이만화가대단하다',
 '#넷플릭스',
 '#일본만화대상',
 '#인생후반전',
 '#올해의책',
 '#인문위클리레터에소개된책',
 '#인생선배',
 '#내맘대로올해의책',
 '#이달의굿즈',
 '#분철',
 '#2023을예측하다',
 '#올해의책',
 '#청년의날추천도서',
 '#크레마클럽에있어요',
 '#심리학처방전',
 '#마음공부',
 '#초등한자',
 '#급수한자',
 '#재밌는한자',
 '#너내동료가돼라',
 '#니체의말',
 '#철학자의말',
 '#서양철학',
 '#철학연구',
 '#분철',
 '#분철',
 '#크레마클럽에있어요',
 '#마법의물건',
 '#초등미스터리',
 '#환상동화',
 '#어린이경제교육',
 '#학습만화',
 '#분철',
 '#크레마클럽에있어요',
 '#명문장',
 '#생활철학',
 '#철학자의인생론',
 '#재미있는철학',
 '#끌어당김',
 '#인생지침서',
 '#올해의책',
 '#성공하고싶다면',
 '#크레마클럽에있어요',
 '#독서지도안',
 '#공부는왜할까?',
 '#북클러버의선택',
 '#일생의기술',
 '#공부법',
 '#분철',
 '#인생후반전',
 '#논어를읽다',
 '#인문위클리레터에소개된책',
 '#크레마클럽에있어요',
 '#나이인문학',
 '#알쓸인잡',
 '#청년의날추천도서',
 '#인문위클리레터에소개된책',
 '#세계여성의날',
 '#여성의날',
 '#유튜버',
 '#학습만화',
 '#부모와아이가행복한육아',
 '#부모교육',


In [42]:
soup.select_one(".img_item>span>a>em>img")["data-original"]

'https://image.yes24.com/goods/97255028/L'

In [28]:
df2['detail_link'][0].split('/')[-1]

KeyError: 'detail_link'

In [ ]:
len(result)

In [ ]:
colnames = ['yy', 'mon', 'title_h', 'title_f', 'title_m', 'title_e', 'detail_link', 'author', 'publisher',
            'pub_date', 'price', 'n_reviews', 'review_link', 'rating', 'tags']

In [ ]:
df = pd.DataFrame(result, columns= colnames)

In [ ]:
df.to_csv("./data/new_yes24bestseller2023_01_1000.csv", index=False)

In [ ]:
df

In [ ]:
data = pd.read_csv("./data/new_yes24bestseller2023_01_1000.csv")

In [ ]:
data.info()

In [ ]:
data['detail_link']

In [ ]:
print(type(data['detail_link']))

In [ ]:
urls = data['detail_link']

In [ ]:
# 데이터 베이스에 저장하기
# 파일로 바로 저장하기

## 상세 페이지 수집

In [ ]:
detail_result.columns

In [ ]:
soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td")

In [ ]:
detail_result.head(2)

In [ ]:
detail_result.columns

In [ ]:
detail_result.info()

# 데이터 베이스에 자료 저장하기
* pandas 데이터 프레임을 DB에 바로 저장하기

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect('sqlite3_yes24_best2023_01.db')
cursor = con.cursor()

In [ ]:
cursor.execute(f'CREATE TABLE IF NOT EXIST yes24_best_202301({detail_result.colums})')
con.commit()

In [ ]:
detail_result['page'] = detail_result['page'].apply(str)

In [ ]:
detail_result['weight'] = detail_result['weight'].apply(str)

In [ ]:
detail_result['category'] = detail_result['category'].apply(str)

In [ ]:
detail_result.to_sql('yes24_best_202301', con, if_exists='append', index=False)

In [ ]:
cursor.execute(
    '''
    SELECT * FROM yes24_best_202301 limit 3;
    ''')
for row in cursor.fetchall():
    print(row)

# sqlalchemy를 이용하는 방법

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlite3_yes24_best2023_01_sqlalchemy.db', echo= False)
conn = engine.raw_connection()
test = pd.read_sql('select * from yes24_best_202301_sqlalchemy limit 3', conn)
test
conn.close()

In [ ]:
def ramentop20todb(df, today):
    engine = create_engine("mysql+mysqldb://<id>:"+"<password>"+"@59.7.246.88:3306/ramen", encoding='utf-8')
    conn = engine.connect()
    try:
        df.to_sql(name='Top20s{}'.format(today), con=engine, if_exists='append', index=False)
    except Exception as e:
        print(e)
    conn.close()
    return print("DB saving has been completed")